In [1]:
import collections
from tqdm import tqdm
import csv

In [2]:
# вытаскиваем предложения из файлов (можно только первые n),
# оставляя или нет тестовую выборку
def get_sents(n=-1, test=True):
    list_of_conllus = ["./syntagrus/ru_syntagrus-ud-dev.conllu",
                       "./syntagrus/ru_syntagrus-ud-train.conllu"]
    if test:
        list_of_conllus.append("./syntagrus/ru_syntagrus-ud-test.conllu")

    sents = []
    i = 0
    for conllu_file in list_of_conllus:
        with open(conllu_file, encoding='utf-8') as f:
            lines = f.readlines()

        sents_one_file = []
        for line in lines:
            if line == "\n":
                s.append(("#", "_")) # маркер конца предложения
                sents_one_file.append(s)
                i += 1
                if i == n:
                    break
            elif line.startswith('# text'): 
                s = [("#", "_")] # маркер начала предложения
            elif not(line.startswith('# sent_id')):
                word = line.split('\t')
                if '.' not in word[0]: # удалила строчки-подпорки для опущенных слов, т.к. они очень мешают дальше, а для нашей задачи пользы не несут
                    pos_word = word[3]
                    num_head = word[6]
                    if all([(num_head != "_"), (num_head != "0")]):
                        num_head = str(int(num_head) + 1)
                    s.append((pos_word, num_head)) # добавляем чр, номер хоста (номер слова, словоформу -- word[0], word[1])
        
        sents.extend(sents_one_file)

    return sents    

**Как записывать n-граммы**
- [ЧР, номер главного(0-3, _), чр главного]
- [{вершина}: [её зависимые]] -- не подходит, тк ничего не скажем про слова n-граммы, у которых главное вне 
- [{(0-3): ЧР}, {(0-3, _ ), чр главного}] -- тоже можно, но так больше "вложенность"

In [3]:
# вытаскиваем n-граммы из предложения
def get_n_grams(n, sent): 
    n_grams = []
    for i in range(n, len(sent) + 1):
        slice_ = sent[i - n : i]
        gram = []
        for word in slice_:
            pos_word = word[0]
            num_head = word[1]
            if num_head == '_' or int(num_head) not in range(i + 1 - n, i + 1):
                num_head = '_'
                pos_head = '_'
            else:
                pos_head = sent[int(num_head) - 1][0]
                num_head = int(num_head) - (i - n)
            gram.append((pos_word, str(num_head), pos_head))
        n_grams.append(tuple(gram))
    return n_grams

In [4]:
# Получаем частотный список
# без тех, в которых есть иностранное слово (pos_word == "X")
# И сколько всего сочетаний
# Конструкции со всеми связяит снаружи оставляем!
def get_count_clear(n, test=True):
    total_n_grams = []
    for sent in tqdm(get_sents(test=test)):
        total_n_grams.extend(get_n_grams(n, sent))

    count_all = collections.Counter(total_n_grams).most_common()
    count_clear = {}
    for n_gram, value in count_all:
        num_foreign = 0
        for word in n_gram:
            if word[0] == "X":
                num_foreign += 1
        if num_foreign == 0: # num_None != n and
            count_clear[n_gram] = value
    
    return count_clear

In [5]:
# Создаем список строк для последующей записи с помощью модуля csv
def prep_for_write(count_clear, keys):
    write_list = []
    
    for col, num in count_clear.items():
        col_list = []
        for c in col:
            col_list.extend(c)
        col_list.append(num)
        
        col_dict = dict()
        for i, key in enumerate(keys):
            col_dict[key] = col_list[i]
        write_list.append(col_dict)
    
    return write_list

In [6]:
# Создаем список ключей-шапка csv таблицы
def generate_keys(n):
    keys = []
    for i in range(1, n+1):
        keys.extend([f"POS{i}", f"#host{i}", f"POShost{i}"])
    keys.append("total_entries")
    return keys

In [7]:
# Выполним все действия и запишем результат в файл
def run_and_write_n_grams(n, file_path, test=True):
    keys = generate_keys(n)
    write_list = prep_for_write(get_count_clear(n, test=test), keys)
    with open(file_path, "w", newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        writer.writerows(write_list)


In [8]:
for i in range(3,7):
    run_and_write_n_grams(i, f"./outcome files/dev_train_test/all_start_finish/all_count_{str(i)}_grams.csv")

100%|██████████████████████████████████████████████████████████████████████████| 61889/61889 [00:19<00:00, 3136.28it/s]


In [9]:
for i in range(3,7):
    run_and_write_n_grams(i, f"./outcome files/dev_train/all_start_finish/all_count_{str(i)}_grams.csv", test=False)

100%|██████████████████████████████████████████████████████████████████████████| 55398/55398 [00:16<00:00, 3375.43it/s]


**Топ-5 для файла dev для 4-грамм**


[((('1', 'ADP', '3', 'NOUN'),
   ('2', 'ADJ', '3', 'NOUN'),
   ('3', 'NOUN', '\_', '\_'),
   ('4', 'PUNCT', '\_', '\_')),
  585),
  
  
 ((('1', 'NOUN', '\_', '\_'),
   ('2', 'ADJ', '3', 'NOUN'),
   ('3', 'NOUN', '1', 'NOUN'),
   ('4', 'PUNCT', '\_', '\_')),
  527),
  
  
 ((('1', 'VERB', '\_', '\_'),
   ('2', 'ADP', '3', 'NOUN'),
   ('3', 'NOUN', '1', 'VERB'),
   ('4', 'PUNCT', '\_', '\_')),
  359),
  
  
 ((('1', 'VERB', '\_', '\_'),
   ('2', 'ADP', '4', 'NOUN'),
   ('3', 'ADJ', '4', 'NOUN'),
   ('4', 'NOUN', '1', 'VERB')),
  323),
  
  
 ((('1', 'ADJ', '3', 'NOUN'),
   ('2', 'ADJ', '3', 'NOUN'),
   ('3', 'NOUN', '\_', '\_'),
   ('4', 'PUNCT', '\_', '\_')),
  309)]

**Примеры и контрпримеры:**

1. *[Кот спит] в большой комнате.*  -- кажется, сломать нельзя, это PP (если правильно понимаю, №1 из статьи про КоСиКо). Единственное -- не сломается ли о *[пришел] в приемную гражданин*? Думаю, не должно, если мы сначала снимем тут частеречную оминимию за счет согласования.
2. *[Вот] хозяйка серого кота.* -- точно нет, потому что *[Она принесла в] коробке серого кота* -- **НО! Кот всё равно серый, эта связь сохраняется**
3. *[Кот] спит на диване.* -- частный вариант шаблона №3 из the статьи (там после РР не обязательно PUNCT)
4. *[Кот] спит на большом диване.* -- то же самое
5. *[Вот] красивый серый кот.* -- ломается о *[Покормила хозяйка] заботливая серого кота.*, хотя это уже какая-то "сказочная" речь -- **Если таких вариантов произношения меньше 2%, то берём**

**Мини-выводы**

1. Разнообразных комбинаций очень много, глазами/руками точно всё не проверить
2. Удаление n-грамм со всеми внешними связми несильно помогло
3. Но при этом, кажется, мы не можем просто выбросить редкие n-граммы: вдруг именно благодаря их редкости там однозначно определяются связи
4. Некоторые комбинации входят в один большой шаблон, но, кажется, заранее это никак не отсечь



**Вопросы**
- какая информация, помимо части речи, нужна? грамматика? словоформа/лемма?
- почему предлог зависит от существительного, а не наоборот?
- почему тут путь к файлу **content/ru_syntagrus-ud-dev.conllu**, хотя сбоку показывается, что файл лежит в папке **sample_data**

**Что еще тут можно сделать:**
- Добавить остальные файлы conllu из syntagrus
- Посмотреть для 5 и 6-грамм (готовая функция уже есть)
- Попробовать посмотреть без знаков препинания (не уверена, что станет лучше, а не хуже, ведь они очень помогают разграничивать некоторые синтаксические группы)

**Fist-look сравнение 4-, 5- и 6-грамм:**
Чем больше окно, тем...
- меньше анализируемых сочетаний
- больше видов конструкций (4 - 64.713, 5 - 213.825, 6 - 416.747)
- меньше частотных конструкций (больше 50 вхождений 4 - 4.3 %, 5 - 0.96 %, 6 - 0.11 %)
- меньше частотность самого частотного (4 - 6073, 5 - 1378, 6 - 371)